In [1]:
from pathlib import Path

import pandas as pd
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport import ForcingParameter, ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization.cost_function.simple_rmse_cost_function import (
    DayCycle,
    SimpleRootMeanSquareErrorCostFunction,
    TimeSeriesObservation,
)
from seapopym_optimization.functional_group import NoTransportFunctionalGroup, Parameter
from seapopym_optimization.genetic_algorithm import SimpleGeneticAlgorithm, SimpleGeneticAlgorithmParameters
from seapopym_optimization.genetic_algorithm.simple_logbook import generate_logbook_with_sobol_sampling
from seapopym_optimization.model_generator import NoTransportModelGenerator


In [ ]:
path_to_forcing = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_cmems_climato.zarr"
path_to_npp = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_observed_npp_climato.zarr"
path_to_obs = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_obs_zoo_climato_monthly_2002_2015.zarr"
export_file_name = "SeapoPym_HOT_climato_obs_npp_opti_all_parameters_1_group_night"

In [ ]:
LATITUDE = 22.75
LONGITUDE = -158
TIME_START = "2005-01-02"
TIME_END = "2009-12-27"
STABILIZATION_TIME = 5
SAVE = True

## Loading


### Forcing


In [ ]:
forcing = xr.open_zarr(path_to_forcing)
forcing = forcing.sel(time=slice(TIME_START, TIME_END))
forcing["T"].attrs["units"] = StandardUnitsLabels.temperature.units
forcing.load()

### Epipelagic layer


In [ ]:
epi_layer_depth = forcing["pelagic_layer_depth"].sel(depth=0).load()
epi_layer_depth = epi_layer_depth.resample(time="1D").mean()
epi_layer_depth.attrs["units"] = "meter"
epi_layer_depth = epi_layer_depth.pint.quantify()
epi_layer_depth

<!-- ## Observed NPP -->


In [ ]:
observed_npp = xr.open_zarr(path_to_npp)
observed_npp = observed_npp.sel(time=slice(TIME_START, TIME_END))
observed_npp = observed_npp.dropna("time", how="all")
observed_npp = observed_npp.resample(time="D").interpolate("linear")
observed_npp.load()

### Observations


In [ ]:
observations = xr.open_zarr(path_to_obs).load()
observations = observations.sel(latitude=LATITUDE, longitude=LONGITUDE, method="nearest")
observations = observations.resample(time="1D").mean().dropna("time")
observations = observations.pint.quantify().pint.to("mg/m^3")
observations = observations * epi_layer_depth
observations = observations.drop_vars("depth")
observations

observations


Select the kind of observation you want to use.


In [ ]:
# observations_selected = observations[["day_lowess_0.2", "night_lowess_0.2"]].rename(
#     {"day_lowess_0.2": "day", "night_lowess_0.2": "night"}
# )

observations_selected = observations["night"]

Remove the X first months to let the model reach the stationary state.


In [ ]:
observations_selected_without_init = observations_selected.isel(time=slice(STABILIZATION_TIME, None))
observations_selected_without_init

Create structure for SeapoPym simulation.


In [ ]:
forcing_parameters = ForcingParameter(
    temperature=ForcingUnit(forcing=forcing["T"]), primary_production=ForcingUnit(forcing=observed_npp["l12"])
)

## Setup the parameters and the cost function


In [ ]:
functional_groups = [
    NoTransportFunctionalGroup(
        name="Zooplankton",
        day_layer=0,
        night_layer=0,
        energy_transfert=Parameter("D1N1_energy_transfert energy_transfert", 0.001, 0.5),
        gamma_tr=Parameter("D1N1_gamma_tr", -0.5, -0.001),
        tr_0=Parameter("D1N1_tr_0", 0, 50),
        gamma_lambda_temperature=Parameter("D1N1_gamma_lambda_temperature", 0, 0.5),
        lambda_temperature_0=Parameter("D1N1_lambda_temperature_0", 0, 1),
    ),
]

In [ ]:
model_generator = NoTransportModelGenerator(forcing_parameters=forcing_parameters)

In [ ]:
cost_function = SimpleRootMeanSquareErrorCostFunction(
    model_generator=model_generator,
    observations=[
        TimeSeriesObservation(
            name="Hot climato",
            observation=observations_selected_without_init,
            observation_type=DayCycle.NIGHT,
            observation_interval="1MS",
        )
    ],
    functional_groups=functional_groups,
    root_mse=True,
    normalized_mse=True,
    centered_mse=False,
)

Set the genetic algorithm meta parameters.


In [ ]:
genetic_algo_parameters = SimpleGeneticAlgorithmParameters(
    MUTPB=0.30,
    INDPB=0.2,
    ETA=5,
    CXPB=0.7,
    NGEN=5,
    POP_SIZE=200,
    cost_function_weight=(-1,),
)

Import or create the parameters to be optimized.


In [ ]:
logbook_path = Path(f"./{export_file_name}_logbook.parquet")
if logbook_path.exists():
    logbook = pd.read_parquet(logbook_path)
else:
    logbook = generate_logbook_with_sobol_sampling(
        functional_group_parameters=functional_groups, sample_number=256, fitness_name=["Hot climato"]
    )
logbook

Finaly, create the Genetic Algorithm.


In [ ]:
client = Client()
genetic_algo = SimpleGeneticAlgorithm(
    meta_parameter=genetic_algo_parameters,
    cost_function=cost_function,
    client=client,
    logbook=logbook,
    save=logbook_path,
)

And watch the magic on the Dask dashboard :


In [ ]:
genetic_algo.client

## Run the optimization


In [ ]:
viewer = genetic_algo.optimize()

## Optimization statistics


In [ ]:
viewer.hall_of_fame().iloc[:50]

In [ ]:
viewer.fitness_evolution(points="all")

In [ ]:
viewer.parameters_standardized_deviation()

In [ ]:
viewer.shannon_entropy(bins=100_000)

In [ ]:
fig = viewer.box_plot(3, nbest=500)
fig.show()

In [ ]:
fig = viewer.parallel_coordinates(nbest=500, reversescale=True, unselected_opacity=0.05, uniformed=True)

for group in fig:
    display(group)